<p style="text-align: center"> Escuela Técnica Superior de Ingeniería Informática</p>
<p style="text-align: center">Universidad de Sevilla</p>
<p style="text-align: center">Procesamiento del Lenguaje Natural </p>
<p style="text-align: center"> Tarea 3 </p>
<p> </p>

**Nombre del alumno:**
- Kenny Jesús Flores Huamán

<!-- 
Para generar la tabla de contenidos que se va a haber a continuación, se ha hecho uso del siguiente código
URL : https://stackoverflow.com/questions/21151450/how-can-i-add-a-table-of-contents-to-a-jupyter-jupyterlab-notebook#:~:text=Click%20the%20toc2%20symbol%20in,you%20open%20it%20in%20JupyterLab.

import json
import urllib
def generate_toc(notebook_path, indent_char="&emsp;"):
    is_markdown = lambda it: "markdown" == it["cell_type"]
    is_title = lambda it: it.strip().startswith("#") and it.strip().lstrip("#").lstrip()
    with open(notebook_path, 'r') as in_f:
        nb_json = json.load(in_f)
    for cell in filter(is_markdown, nb_json["cells"]):
        for line in filter(is_title, cell["source"]):
            line = line.strip()
            indent = indent_char * (line.index(" ") - 1)
            title = line.lstrip("#").lstrip()
            url = urllib.parse.quote(title.replace(" ", "-"))
            out_line = f"{indent}[{title}](#{url})<br>\n"
            print(out_line, end="")
            
generate_toc('C6_FloresHuaman.ipynb') -->


# Introduccion 

En esta tarea vamos a hacer uso de la herramienta jupyter notebook para profundizar algunos aspectos del procesamiento del lenguaje natural, en este caso vamos a realizar una predicción de la siguiente palabra que escribamos.


# Librerias importadas

In [21]:
# !pip install pandas
# !pip install nltk
import pandas as pd
import numpy as np
import string
import re
import pickle
import nltk
from nltk.corpus import stopwords
# nltk.download("stopwords")

In [22]:
# Cargamos las stopwords en español
stopwords = set(stopwords.words('spanish'))

## Dataset utilizado

En esta ocasión, debido a los nulos datasets relacionados con los chats en español, se ha decidido elegir un dataset donde recopilan [+9000 letras de rap en español](https://www.kaggle.com/datasets/smunoz3801/9325-letras-de-rap-en-espaol).



In [60]:
dataset = pd.read_csv("dataset.csv")

Debido a que de todo el dataset lo único que nos interesa es el corpus, vamos a convertirlo en un array numpy

In [61]:
letras = dataset['letra'].values.astype('U')

Generamos una máscara para poder eliminar las letras que no tienen letra.

In [ ]:
mask = np.char.find(letras, '¿Tienes ya la letra para este tema? Ayúdanos y ¡Envíanosla!') == -1
letras = letras[mask]

In [68]:
letras_limpio = []
# Eliminamos canciones sin letra
for letra in list(letras):
    if "¿Tienes ya la letra para este tema? Ayúdanos y ¡Envíanosla!" in letra:
        letras.remove(letra)

#Eliminamos líneas vacías o con [Artista] o [Estribillo]
for i in range(len(letras)):
    cancion_limpia = []
    for linea in letras[i].split("\n"):
        if ("[" not in linea and "(" not in linea and linea != ""):
            #Pasamos línea a minúsculas y eliminamos puntuación
            linea = bytes(linea, 'utf-8').decode('utf-8', 'ignore')
            linea = "".join(c for c in linea if (c not in string.punctuation and c not in ['','¡','¿'])).lower()
            linea = linea.split(" ")
            #Eliminamos stopwords
            for palabra in list(linea):
                #palabra = palabra.replace(u'\xa0', u'') #Estp les pasa por usar latin en vez de UTF-8
                if palabra in stopwords or palabra in string.punctuation:
                    linea.remove(palabra)
            cancion_limpia += linea
    letras_limpio += [cancion_limpia]